In [124]:
import sys
import datetime as dt

import pandas as pd
import numpy as np
from IPython.display import display, HTML

sys.path.append('../invoicing')
from main import *

%load_ext autoreload
%autoreload 2

data_dir = '../tests/data/'



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [117]:
path = data_dir + 'timesheet.csv'
timesheet = read_timesheet(path, time_units='min')
timesheet

,date,project,time_spent,comment
0,2016-04-01,A,4.000000,NaN
1,2016-04-04,B,2.000000,NaN
2,2016-04-04,C,1.083333,NaN
3,2016-04-06,A,10.000000,NaN
4,2016-04-13,A,5.333333,booyah
5,2016-04-14,D,12.233333,doh
6,2016-04-15,A,8.350000,NaN


In [118]:
f = slice_to_dates(timesheet, '20160401', '20160402')
f

,date,project,time_spent,comment
0,2016-04-01,A,4.0,NaN


In [120]:
pr_df = pd.DataFrame([
  ['A', 80],
  ['B', 100],
  ['C', 160],
  ['D', 100],
  ], columns=['project', 'rate'])

f = bill_at_project_rates(timesheet, pr_df, '20160401', '21040402')
f

,project,time_spent,rate,cost
0,A,27.683333,80,2214.666667
1,B,2.000000,100,200.000000
2,C,1.083333,160,173.333333
3,D,12.233333,100,1223.333333


In [139]:
tr_df = pd.DataFrame([
  [10, 1, 30],
  [20, 2, 60],
  [30, 3, 100],
  [np.inf, 4, 200],        
  ], columns=['time_cutoff', 'tier', 'rate'])
tr_df

def bill(timesheet, tiered_rates_df, freq='W', 
  groupby_project=False, date1=None, date2=None):
    # Slice
    timesheet = slice_by_dates(timesheet, date1, date2)

    # Resample to frequency by summing
    if groupby_project:
        f = timesheet.groupby('project').apply(
          lambda x: x.set_index('date')[['time_spent']].resample('W').sum())
    else:
        f = timesheet.set_index('date')[['time_spent']].resample('W').sum()
    
    # Partition each group by tiered rates
    def my_agg(group):
        t = group['time_spent'].iat[0]
        return partition_by_tiered_rates(t, tiered_rates_df)

    f = f.reset_index().rename(columns={'date': 'period_end'})
    
    if groupby_project:
        cols = ['period_end', 'project']
        drop_cols = ['level_2']
    else:
        cols = ['period_end']
        drop_cols = ['level_1']
        
    g = f.groupby(cols).apply(my_agg).reset_index().drop(drop_cols, axis=1)

    # Compute cost
    g['cost'] = g['time_spent']*g['rate']

    return g

bill(timesheet, tr_df, groupby_project=True)

,period_end,project,time_cutoff,tier,rate,time_spent,cost
0,2016-04-03,A,10.000000,1,30,4.000000,120.0
1,2016-04-03,A,20.000000,2,60,0.000000,0.0
2,2016-04-03,A,30.000000,3,100,0.000000,0.0
3,2016-04-03,A,inf,4,200,0.000000,0.0
4,2016-04-10,A,10.000000,1,30,10.000000,300.0
5,2016-04-10,A,20.000000,2,60,0.000000,0.0
6,2016-04-10,A,30.000000,3,100,0.000000,0.0
7,2016-04-10,A,inf,4,200,0.000000,0.0
8,2016-04-10,B,10.000000,1,30,2.000000,60.0
9,2016-04-10,B,20.000000,2,60,0.000000,0.0


NameError: name 'bill_by_tiered_rates' is not defined

In [115]:
html = make_invoice(timesheet)


In [11]:
display(HTML(html))

From:,Alex,To:,
Project,Time (h),Rate (NZD/h),Total (NZD)
Bingo,2,150,300
